# Expedia - Recommender

# Stappenplan
  * Missing data: 1 column
  * Log transformations
  * Outliers
  * selection - investigate correlation with target variable
  * combine columns
  * make new columns: datediff
  * Scaling
  * One-hot encoding
  * PCA
  * search attributes
  * Try different classification models

type bookers
week of weekend
zakelijk of prive
mobile en low distance
package
use past records of user: laatste keer geboekt, aantal keer geboekt tot nu toe
lengte vakantie

# Import packages

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
from pandas.plotting import scatter_matrix

from scipy import stats
from collections import Counter
from math import sqrt
import os

import datetime

from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.feature_selection import RFECV
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV, Lasso, LassoCV, ElasticNet, LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score, r2_score, mean_squared_error

from ml_metrics import apk, mapk

# Create functions

In [127]:
def countplot(df, var):
    fig, ax = plt.subplots(figsize=(25,3))
    sns.countplot(x=df[var], ax=ax)
    ax.set_title(f"Distribution {var}")

In [128]:
def get_mean_average_precision(predicted, correct, cutoff=5):
    average_precision = []
    for i, cluster in enumerate(correct):
        average_precision.append(int(cluster in predicted[i][:cutoff]) / len(predicted[i][:cutoff]))
    return np.mean(average_precision)

# Set variables

In [3]:
#"C:/Users/Karli/Documents/XcceleratedAssessment/Data science assessment/assessment/data/"
data_path = "../Data/"
os.chdir(data_path)

pd.set_option('display.max_columns', None) 
pd.set_option('display.float_format', lambda x: '%.5f' % x)
#pd.set_option('display.max_rows', 500)

# Load data

In [6]:
destinations = pd.read_csv('destinations.csv')

!pip install --user --upgrade pandas

In [4]:
train = pd.read_csv('train.csv')#, nrows=10000

In [10]:
train_bookings = train[train['is_booking'] == 1]

In [11]:
test = pd.read_csv('test.csv')
sample_submission = pd.read_csv('sample_submission.csv')

## Destinations

Consists of features extracted from hotel reviews text

* d1-d149: latent description of search regions	
* srch_destination_id: ID of the destination where the hotel search was performed

In [12]:
destinations.shape

(62106, 150)

In [13]:
destinations.head()

,srch_destination_id,d1,d2,d3,d4,d5,d6,d7,d8,d9,d10,d11,d12,d13,d14,d15,d16,d17,d18,d19,d20,d21,d22,d23,d24,d25,d26,d27,d28,d29,d30,d31,d32,d33,d34,d35,d36,d37,d38,d39,d40,d41,d42,d43,d44,d45,d46,d47,d48,d49,d50,d51,d52,d53,d54,d55,d56,d57,d58,d59,d60,d61,d62,d63,d64,d65,d66,d67,d68,d69,d70,d71,d72,d73,d74,d75,d76,d77,d78,d79,d80,d81,d82,d83,d84,d85,d86,d87,d88,d89,d90,d91,d92,d93,d94,d95,d96,d97,d98,d99,d100,d101,d102,d103,d104,d105,d106,d107,d108,d109,d110,d111,d112,d113,d114,d115,d116,d117,d118,d119,d120,d121,d122,d123,d124,d125,d126,d127,d128,d129,d130,d131,d132,d133,d134,d135,d136,d137,d138,d139,d140,d141,d142,d143,d144,d145,d146,d147,d148,d149
0,0,-2.19866,-2.19866,-2.19866,-2.19866,-2.19866,-1.89763,-2.19866,-2.19866,-1.89763,-2.19866,-2.19866,-2.19866,-2.19866,-2.19866,-2.19866,-1.89763,-2.19866,-2.19866,-2.19866,-2.19866,-2.19866,-2.19866,-2.19866,-2.19866,-2.19866,-2.19866,-2.19866,-2.19866,-2.19866,-2.19866,-2.19866,-2.19866,-2.19866,-2.19866,-2.19866,-2.19866,-1.89763,-2.19866,-2.19866,-2.19866,-2.19866,-2.19866,-1.89763,-2.19866,-2.19866,-2.19866,-2.19866,-2.19866,-2.19866,-2.19866,-2.19866,-2.19866,-2.19866,-2.19866,-2.19866,-2.19866,-2.19866,-2.19866,-2.19866,-2.19866,-2.19866,-2.19866,-2.19866,-2.19866,-2.19866,-2.19866,-2.19866,-2.19866,-2.19866,-2.19866,-2.19866,-2.19866,-2.19866,-2.19866,-2.19866,-2.19866,-2.19866,-2.19866,-2.19866,-2.19866,-2.19866,-2.19866,-2.19866,-2.19866,-2.19866,-2.19866,-2.19866,-2.19866,-2.19866,-2.19866,-2.19866,-2.19866,-1.89763,-2.19866,-2.19866,-2.19866,-2.19866,-2.19866,-2.19866,-2.19866,-2.19866,-2.19866,-2.19866,-2.19866,-2.19866,-2.19866,-2.19866,-2.19866,-2.19866,-2.19866,-2.19866,-2.19866,-2.19866,-2.19866,-2.19866,-2.19866,-2.19866,-2.19866,-2.19866,-2.19866,-1.89763,-2.19866,-2.19866,-2.19866,-2.19866,-2.19866,-2.19866,-1.89763,-2.19866,-2.19866,-2.19866,-1.89763,-2.19866,-2.19866,-2.19866,-2.19866,-2.19866,-2.19866,-2.19866,-2.19866,-2.19866,-2.19866,-2.19866,-2.19866,-2.19866,-2.19866,-2.19866,-2.19866,-2.19866
1,1,-2.18169,-2.18169,-2.18169,-2.08256,-2.18169,-2.16503,-2.18169,-2.18169,-2.03160,-2.18169,-2.18169,-2.18169,-2.18169,-2.18169,-2.18169,-2.18169,-2.18169,-2.18169,-2.18169,-2.18169,-2.18169,-2.18169,-2.18169,-2.18169,-2.18169,-2.18169,-2.18169,-2.18169,-2.18169,-2.18169,-2.18169,-2.18169,-2.18169,-2.18169,-2.18169,-2.18169,-2.04379,-2.18169,-2.18169,-2.18169,-2.16503,-2.18169,-2.13351,-2.18169,-2.18169,-2.18169,-2.18169,-2.18169,-2.18169,-2.18169,-2.18169,-2.18169,-2.13351,-2.18169,-2.18169,-2.18169,-2.18169,-2.08256,-2.18169,-2.18169,-2.18169,-2.18169,-2.18169,-2.18169,-2.18169,-2.18169,-2.18169,-2.18169,-2.18169,-2.18169,-2.18169,-2.18169,-2.18169,-2.18169,-2.18169,-2.18169,-2.18169,-2.18169,-2.18169,-2.18169,-2.18169,-2.18169,-2.18169,-2.16503,-2.16503,-2.18169,-2.18169,-2.18169,-2.18169,-2.18169,-2.08256,-2.18169,-2.13351,-2.18169,-2.18169,-2.18169,-2.18169,-2.08256,-2.18169,-2.18169,-2.18169,-2.18169,-2.16503,-2.18169,-2.18169,-2.18169,-2.18169,-2.14898,-2.18169,-2.18169,-2.18169,-2.18169,-2.18169,-2.18169,-2.18169,-2.18169,-2.18169,-2.18169,-2.18169,-2.18169,-2.18169,-2.13351,-2.18169,-2.18169,-2.18169,-2.18169,-2.18169,-2.13351,-2.18169,-2.18169,-2.18169,-2.14898,-2.18169,-2.18169,-2.18169,-2.18169,-2.14898,-2.18169,-2.18169,-2.16503,-2.18169,-2.16503,-2.18169,-2.18169,-2.16503,-2.18169,-2.18169,-2.18169,-2.18169
2,2,-2.18349,-2.22416,-2.22416,-2.18956,-2.10582,-2.07541,-2.22416,-2.11848,-2.14039,-2.22416,-2.20985,-2.22416,-2.11072,-2.18601,-2.22416,-2.12447,-2.22416,-2.15647,-2.22416,-2.22416,-2.22416,-2.22416,-2.22416,-2.22416,-2.22416,-2.22416,-2.22416,-2.22416,-2.15332,-2.18601,-2.22416,-2.22416,-2.22416,-2.12824,-2.15831,-2.05972,-1.91027,-2.22416,-2.12305,-2.22416,-2.15405,-2.19733,-1.94546,-2.22416,-2.22416,-2.19733,-2.16689,-2.19201,-1.90378,-2.22416,-2.18956,-2.22416,-2.08310,-2.22416,-2.07010,-2.15774,-2.19638,-2.06981,-2.22416,-2.22416,-2.07295,-2.22416,-2.22416,-2.22416,-2.22416,-2.14039,-2.22416,-2.22416,-2.09655,-2.22416,-2.18753,-2.22416,

## Train

The train and test datasets are split based on time: training data from 2013 and 2014, while test data are from 2015. The public/private leaderboard data are split base on time as well. Training data includes all the users in the logs, including both click events and booking events. Test data only includes booking events. 

In [14]:
train_bookings.shape

(3000693, 24)

In [15]:
train_bookings.head()

,date_time,site_name,posa_continent,user_location_country,user_location_region,user_location_city,orig_destination_distance,user_id,is_mobile,is_package,channel,srch_ci,srch_co,srch_adults_cnt,srch_children_cnt,srch_rm_cnt,srch_destination_id,srch_destination_type_id,is_booking,cnt,hotel_continent,hotel_country,hotel_market,hotel_cluster
1,2014-08-11 08:22:12,2,3,66,348,48862,2234.26410,12,0,1,9,2014-08-29,2014-09-02,2,0,1,8250,1,1,1,2,50,628,1
20,2014-02-27 18:01:32,2,3,66,318,52078,nan,756,0,1,4,2014-04-18,2014-04-20,2,0,1,8291,1,1,1,2,50,191,2
27,2013-06-15 15:38:05,30,4,195,548,56440,nan,1048,0,1,9,2013-09-06,2013-09-14,2,0,1,1385,1,1,1,0,185,185,58
72,2014-11-23 18:02:20,30,4,195,991,47725,nan,1048,0,0,9,2015-06-26,2015-06-28,2,0,1,8803,1,1,1,3,151,69,36
79,2014-01-03 16:30:17,2,3,66,462,41898,2454.85880,1482,0,1,1,2014-02-22,2014-02-27,3,0,2,12009,1,1,1,2,50,680,95


In [16]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37670293 entries, 0 to 37670292
Data columns (total 24 columns):
 #   Column                     Dtype  
---  ------                     -----  
 0   date_time                  object 
 1   site_name                  int64  
 2   posa_continent             int64  
 3   user_location_country      int64  
 4   user_location_region       int64  
 5   user_location_city         int64  
 6   orig_destination_distance  float64
 7   user_id                    int64  
 8   is_mobile                  int64  
 9   is_package                 int64  
 10  channel                    int64  
 11  srch_ci                    object 
 12  srch_co                    object 
 13  srch_adults_cnt            int64  
 14  srch_children_cnt          int64  
 15  srch_rm_cnt                int64  
 16  srch_destination_id        int64  
 17  srch_destination_type_id   int64  
 18  is_booking                 int64  
 19  cnt                        int64  
 20  

In [17]:
train.describe()

,site_name,posa_continent,user_location_country,user_location_region,user_location_city,orig_destination_distance,user_id,is_mobile,is_package,channel,srch_adults_cnt,srch_children_cnt,srch_rm_cnt,srch_destination_id,srch_destination_type_id,is_booking,cnt,hotel_continent,hotel_country,hotel_market,hotel_cluster
count,37670293.00000,37670293.00000,37670293.00000,37670293.00000,37670293.00000,24145292.00000,37670293.00000,37670293.00000,37670293.00000,37670293.00000,37670293.00000,37670293.00000,37670293.00000,37670293.00000,37670293.00000,37670293.00000,37670293.00000,37670293.00000,37670293.00000,37670293.00000,37670293.00000
mean,9.79527,2.68047,86.10880,308.40601,27753.04473,1970.09003,604451.75318,0.13493,0.24890,5.87076,2.02430,0.33212,1.11266,14441.09054,2.58228,0.07966,1.48338,3.15630,81.29685,600.46188,49.80861
std,11.96754,0.74804,59.24310,208.44375,16782.55320,2232.44243,350617.46204,0.34165,0.43238,3.71709,0.91167,0.73150,0.45912,11066.30233,2.15302,0.27076,1.21978,1.62319,56.17119,511.73913,28.91595
min,2.00000,0.00000,0.00000,0.00000,0.00000,0.00560,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000
25%,2.00000,3.00000,66.00000,174.00000,13009.00000,313.16700,298910.00000,0.00000,0.00000,2.00000,2.00000,0.00000,1.00000,8267.00000,1.00000,0.00000,1.00000,2.00000,50.00000,160.00000,25.00000
50%,2.00000,3.00000,66.00000,314.00000,27655.00000,1140.49110,603914.00000,0.00000,0.00000,9.00000,2.00000,0.00000,1.00000,9147.00000,1.00000,0.00000,1.00000,2.00000,50.00000,593.00000,49.00000
75%,14.00000,3.00000,70.00000,385.00000,42413.00000,2552.59930,910168.00000,0.00000,0.00000,9.00000,2.00000,0.00000,1.00000,18790.00000,5.00000,0.00000,2.00000,4.00000,106.00000,701.00000,73.00000
max,53.00000,4.00000,239.00000,1027.00000,56508.00000,12407.90220,1198785.00000,1.00000,1.00000,10.00000,9.00000,9.00000,8.00000,65107.00000,9.00000,1.00000,269.00000,6.00000,212.00000,2117.00000,99.00000


In [18]:
print('There are {} users present in the data.'.format(len(set(train['user_id']))))

There are 1198786 users present in the data.


In [19]:
# Some users have quite some records
Counter(train['user_id']).most_common(10)

[(1187360, 530),
 (1040395, 501),
 (124565, 498),
 (1043120, 495),
 (783124, 491),
 (96700, 490),
 (685462, 485),
 (908612, 483),
 (1011295, 480),
 (52390, 479)]

In [ ]:
# Some customers have many bookings, some don't have any - only clicks. Do we want to throw out all customers that
# only have clicks as we are ultimately only interested in the bookings they make?
Counter(train_bookings['user_id']).most_common(10)

[(966474, 100),
 (602738, 100),
 (911486, 100),
 (235912, 100),
 (913337, 100),
 (625627, 100),
 (1049940, 100),
 (1137579, 100),
 (1017892, 100),
 (944701, 99)]

In [ ]:
train.isna().sum()

## Visualizations

In [ ]:
#countplot(train[train['user_id'] == 527020], 'hotel_cluster')

In [ ]:
# Hotel cluster of one customer
#countplot(train_bookings[train_bookings['user_id'] == 527020], 'hotel_cluster')

In [ ]:
countplot(train, 'hotel_cluster')

In [ ]:
Counter(train_bookings['hotel_cluster']).most_common(10)

In [ ]:
# Hotel cluster of one customer
countplot(train_bookings, 'hotel_cluster')

In [ ]:
countplot(train, 'is_booking')

In [ ]:
# Most user events only have a count of 1 of the similar events in the context of the same user session
countplot(train_bookings, 'cnt')

In [ ]:
countplot(train_bookings[train_bookings['srch_children_cnt'] > 0], 'hotel_cluster')

In [ ]:
countplot(train_bookings[train_bookings['srch_adults_cnt'] == 1], 'hotel_cluster')

In [ ]:
countplot(train_bookings, 'posa_continent')

In [ ]:
countplot(train_bookings, 'site_name')

In [ ]:
# Most bookings are for two adults
countplot(train_bookings, 'srch_adults_cnt')

In [ ]:
# Most look for one room appartments
countplot(train_bookings, 'srch_rm_cnt')

In [ ]:
# Most common to not have any children
countplot(train_bookings, 'srch_children_cnt')

In [ ]:
countplot(train_bookings, 'is_package')

In [ ]:
countplot(train_bookings, 'is_package')

In [ ]:
sns.countplot(x='is_package',hue='is_booking',data=train)

In [ ]:
countplot(train_bookings, 'is_mobile')

In [ ]:
sns.countplot(x='is_mobile',hue='is_booking',data=train)

# Change types

In [ ]:
#train['srch_ci_dt'] = train['srch_ci'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d'))
#pd.to_datetime(train['srch_ci'])

# Compare train and test

In [ ]:
sns.countplot(x='cnt',data=train)

In [ ]:
set(train.columns) - set(test.columns)

In [ ]:
test.head()

In [ ]:
print("Percentage existing users in test set: ")
len(set(train['user_id']) & set(test['user_id'])) / len(set(test['user_id'])) * 100

# First 'model'

In [ ]:
#df = pd.DataFrame({'id': [i for i in range(0,2528243)], 'hotel_cluster': ['91'] * 2528243})
#df

In [ ]:
#df.to_csv('baseline_output.csv', index=False)

# Change types

In [ ]:
train_bookings['srch_ci_dt'] = pd.to_datetime(train_bookings['srch_ci'])
train_bookings['srch_co_dt'] = pd.to_datetime(train_bookings['srch_co'])

# PCA search attributes

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pca = PCA(n_components=2)

In [ ]:
search = [x for x in list(train.columns) if 'srch' in x]
search_df = train[search]

In [ ]:
user = [x for x in list(train.columns) if 'user' in x]
user_df = train[user]

In [ ]:
df = train.groupby('user_id')['user_location_country'].nunique()
len(df[df>1])/len(df)

In [ ]:
len(train[train['orig_destination_distance']<400])/len(train)

In [ ]:
principalComponents_search = pca.fit_transform(user_df)

principalDf = pd.DataFrame(data = principalComponents
             , columns = ['principal component 1', 'principal component 2'])

# Feature engineering

In [ ]:
train.columns

In [ ]:
train.groupby('user_location_city')['hotel_cluster'].nunique()

In [ ]:
train_bookings = train_bookings.set_index('user_id')

In [ ]:
train_bookings['srch_ci_wd'] = train_bookings['srch_ci_dt'].apply(lambda x: x.weekday())
train_bookings['srch_co_wd'] = train_bookings['srch_co_dt'].apply(lambda x: x.weekday())

In [ ]:
train_bookings[['srch_ci_wd','srch_co_wd']]

In [ ]:
train_bookings['srch_total_time'] = (train_bookings['srch_co_dt']-train_bookings['srch_ci_dt']).apply(lambda x: x.days)

In [ ]:
train_bookings['srch_total_time'].hist(bins=30)

In [ ]:
train_bookings[train_bookings['orig_destination_distance'].isna()]

In [ ]:
train_bookings.isna().sum()

In [ ]:
df1 = train_bookings.groupby('hotel_cluster')['is_mobile'].sum()
df1[df1>3]

In [ ]:
df1 = train_bookings.groupby('hotel_cluster')['channel'].count()
len(df1[df1>5])

In [ ]:
df = train_bookings.corr()[['hotel_cluster']]
df[abs(df) > 0.05]

In [ ]:
sns.heatmap(train.corr())

#cnt
#hotel_continent
#srch_adults_cnt
#is_mobile
#is_package
#user_location_region

In [ ]:
sns.heatmap(train_bookings.corr())

In [ ]:
#aantal dagen boeking

# Some what more advanced model

In [ ]:
train.head()

In [ ]:
X = train[['posa_continent','cnt','hotel_continent','srch_adults_cnt','is_mobile', 'is_package','user_location_region']]

In [ ]:
y = train['is_booking']

In [ ]:
clf = LogisticRegression(random_state=0).fit(X, y)

In [ ]:
clf.score(X, y)

# Collaborative filtering

In [ ]:
# Start only with the bookings
user_bookings = (train_bookings[['user_id', 'hotel_cluster']]
                 .groupby('user_id')
                 .agg({'hotel_cluster': list})
                 .reset_index())
user_bookings

In [ ]:
n_users = len(set(train_bookings['user_id']))
print(n_users)
n_clusters = len(set(train_bookings['hotel_cluster']))
print(n_clusters)

In [ ]:
feedback = np.zeros([n_users, n_clusters])

In [ ]:
user_bookings.index

In [ ]:
# Fill feedback matrix
feedback = np.zeros([n_users, n_clusters])
for user in range(len(user_bookings.index)):
    feedback[user, user_bookings.loc[user, 'hotel_cluster']] = 1

In [ ]:
from scipy import spatial

spatial.distance.cosine(feedback[0], feedback[4])

# Evalution

In [ ]:
correct_clusters = np.ones(len(sample_submission)) * 99
predicted_clusters = sample_submission['hotel_cluster'].apply(lambda x: [int(i) for i in x.split(' ')])

In [ ]:
get_mean_average_precision(predicted_clusters, correct_clusters)